In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType, StructField, StringType, IntegerType, TimestampType, LongType
from pyspark.sql.functions import to_json, struct, col  # Corrected import

In [63]:
spark = SparkSession \
        .builder \
        .appName("read-parquet-write-to-kafka") \
        .config("spark.executor.memory", "4g") \
        .config("spark.driver.memory", "4g") \
        .config("spark.executor.cores", "4") \
        .config("spark.executor.instances", "2") \
        .config("spark.sql.shuffle.partitions", "12") \
        .config("spark.jars.packages", "org.apache.spark:spark-sql-kafka-0-10_2.12:3.5.2") \
        .getOrCreate()

In [64]:
df = spark.read.format("csv").option("header", "true").load("work/output/data/chunk_0.csv")

In [65]:
df.printSchema()

root
 |-- ID: string (nullable = true)
 |-- SPEED: string (nullable = true)
 |-- TRAVEL_TIME: string (nullable = true)
 |-- STATUS: string (nullable = true)
 |-- DATA_AS_OF: string (nullable = true)
 |-- LINK_ID: string (nullable = true)
 |-- LINK_POINTS: string (nullable = true)
 |-- ENCODED_POLY_LINE: string (nullable = true)
 |-- ENCODED_POLY_LINE_LVLS: string (nullable = true)
 |-- OWNER: string (nullable = true)
 |-- TRANSCOM_ID: string (nullable = true)
 |-- BOROUGH: string (nullable = true)
 |-- LINK_NAME: string (nullable = true)



In [66]:
SCHEMA = StructType([
    StructField("ID", StringType()),
    StructField("SPEED", StringType()),
    StructField("TRAVEL_TIME", StringType()),
    StructField("STATUS", StringType()),
    StructField("DATA_AS_OF", StringType()),
    StructField("LINK_ID", StringType()),
    StructField("LINK_POINTS", StringType()),
    StructField("ENCODED_POLY_LINE", StringType()),
    StructField("ENCODED_POLY_LINE_LVLS", StringType()),
    StructField("OWNER", StringType()),
    StructField("TRANSCOM_ID", StringType()),
    StructField("BOROUGH", StringType()),
    StructField("LINK_NAME", StringType())
])

In [67]:
# Read the parquet file write it to the topic
# We need to specify the schema in the stream
# and also convert the entries to the format key, value
parquet_input_path = "work/output/parquet/" #"work/output/parquet/*/*.parquet"  # Update this path to where your Parquet files are stored
df = spark.read.parquet(parquet_input_path)

# Display schema to verify the structure
df.take(1)

AnalysisException: [UNABLE_TO_INFER_SCHEMA] Unable to infer schema for Parquet. It must be specified manually.

In [68]:
df.printSchema()

root
 |-- ID: string (nullable = true)
 |-- SPEED: string (nullable = true)
 |-- TRAVEL_TIME: string (nullable = true)
 |-- STATUS: string (nullable = true)
 |-- DATA_AS_OF: string (nullable = true)
 |-- LINK_ID: string (nullable = true)
 |-- LINK_POINTS: string (nullable = true)
 |-- ENCODED_POLY_LINE: string (nullable = true)
 |-- ENCODED_POLY_LINE_LVLS: string (nullable = true)
 |-- OWNER: string (nullable = true)
 |-- TRANSCOM_ID: string (nullable = true)
 |-- BOROUGH: string (nullable = true)
 |-- LINK_NAME: string (nullable = true)



In [69]:
#df = spark.read.schema(SCHEMA).parquet(parquet_input_path)

In [70]:
#df.createOrReplaceTempView("traffic_data")

In [71]:
# kafka_df_traffic_stream = spark.sql(
#     """
#     select 
#         id as key,
#         to_json(struct(*)) as value
#     from traffic_data
#     """
# )

# kafka_df1 = df.selectExpr(
#     "ID as key",
#     "to_json(struct(*)) as value"
# )

In [72]:
#kafka_df.take(1)

In [73]:
#kafka_df1.take(1)

In [74]:
df_traffic_stream = spark.readStream \
    .format("parquet") \
    .option("maxFilesPerTrigger", 5) \
    .schema(SCHEMA) \
    .load(parquet_input_path) \
    .withColumn("value", to_json(struct(col("*")))) \
    .withColumn("key", col("ID").cast("string")) \
    #.limit(10)
    # .withColumn("value", to_json(struct([col(c) for c in df_traffic_stream.columns] ))) \
    # .withColumn("key", col("ID").cast("string")) \
    

In [75]:
# Convert DataFrame to JSON format required for Kafka
# df_traffic_stream = df_traffic_stream \
#     .withColumn("value", to_json(struct([col(c) for c in df_traffic_stream.columns]))) \
#     .withColumn("key", col("ID").cast("string"))

In [76]:
#json_df = df_traffic_stream.selectExpr("to_json(struct(*)) as value")

In [77]:
# query = df_traffic_stream.selectExpr("cast(key as string)", "cast(value as string)").writeStream \
#     .format("console")\
#     .outputMode("append")\
#     .option("truncate", "false") \
#     .start()

In [78]:
query = df_traffic_stream \
    .selectExpr("cast(key as string)", "cast(value as string)") \
    .writeStream \
    .format("kafka") \
    .option("kafka.bootstrap.servers", "kafka:9092") \
    .option("topic", "traffic_sensor") \
    .option("checkpointLocation", "/tmp/spark-kafka-checkpoint") \
    .trigger(processingTime='5 seconds') \
    .start()

In [79]:
query.awaitTermination()

ERROR:root:KeyboardInterrupt while sending command.
Traceback (most recent call last):
  File "/usr/local/spark/python/lib/py4j-0.10.9.7-src.zip/py4j/java_gateway.py", line 1038, in send_command
    response = connection.send_command(command)
               ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/spark/python/lib/py4j-0.10.9.7-src.zip/py4j/clientserver.py", line 511, in send_command
    answer = smart_decode(self.stream.readline()[:-1])
                          ^^^^^^^^^^^^^^^^^^^^^^
  File "/opt/conda/lib/python3.11/socket.py", line 706, in readinto
    return self._sock.recv_into(b)
           ^^^^^^^^^^^^^^^^^^^^^^^
KeyboardInterrupt


KeyboardInterrupt: 

In [16]:
#query.awaitTermination()

ERROR:root:KeyboardInterrupt while sending command.
Traceback (most recent call last):
  File "/usr/local/spark/python/lib/py4j-0.10.9.7-src.zip/py4j/java_gateway.py", line 1038, in send_command
    response = connection.send_command(command)
               ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/spark/python/lib/py4j-0.10.9.7-src.zip/py4j/clientserver.py", line 511, in send_command
    answer = smart_decode(self.stream.readline()[:-1])
                          ^^^^^^^^^^^^^^^^^^^^^^
  File "/opt/conda/lib/python3.11/socket.py", line 706, in readinto
    return self._sock.recv_into(b)
           ^^^^^^^^^^^^^^^^^^^^^^^
KeyboardInterrupt


KeyboardInterrupt: 

In [19]:
# now checked read and attaching of schema. but since we want to simulate streaming of data, we will create readstream
df_traffic_stream = spark.readStream.format("parquet")\
    .schema(SCHEMA)\
    .load(parquet_input_path)\
    .withColumn("value", F.to_json(F.struct(F.col("*")))) \
    .withColumn("key", F.col("ID")) \
    .limit(400000)

In [22]:
df_traffic_stream \
    .selectExpr("CAST(key AS STRING)", "CAST(value AS STRING)") \
    .writeStream \
    .format("console") \
    .outputMode("append") \
    .option("truncate", "false") \
    .start() \
    .awaitTermination()

ERROR:root:Exception while sending command.
Traceback (most recent call last):
  File "/opt/conda/lib/python3.11/site-packages/IPython/core/interactiveshell.py", line 3526, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "/tmp/ipykernel_13410/2929586154.py", line 8, in <module>
    .awaitTermination()
     ^^^^^^^^^^^^^^^^^^
  File "/usr/local/spark/python/pyspark/sql/streaming/query.py", line 221, in awaitTermination
    return self._jsq.awaitTermination()
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/spark/python/lib/py4j-0.10.9.7-src.zip/py4j/java_gateway.py", line 1322, in __call__
    return_value = get_return_value(
                   ^^^^^^^^^^^^^^^^^
  File "/usr/local/spark/python/pyspark/errors/exceptions/captured.py", line 185, in deco
    raise converted from None
pyspark.errors.exceptions.captured.StreamingQueryException: [STREAM_FAILED] Query [id = 11a40d4b-9ca7-4c5b-91f1-dfb0672e94f9, runId = 53152a54-d11b-4dc8-bde8-11364e7ff61e] ter

StreamingQueryException: [STREAM_FAILED] Query [id = 11a40d4b-9ca7-4c5b-91f1-dfb0672e94f9, runId = 53152a54-d11b-4dc8-bde8-11364e7ff61e] terminated with exception: Job aborted due to stage failure: Task 0 in stage 12.0 failed 1 times, most recent failure: Lost task 0.0 in stage 12.0 (TID 1815) (c661253c945d executor driver): java.lang.OutOfMemoryError: Java heap space
	at java.base/java.nio.HeapByteBuffer.<init>(HeapByteBuffer.java:64)
	at java.base/java.nio.ByteBuffer.allocate(ByteBuffer.java:363)
	at org.apache.spark.serializer.SerializerHelper$.$anonfun$serializeToChunkedBuffer$1(SerializerHelper.scala:40)
	at org.apache.spark.serializer.SerializerHelper$.$anonfun$serializeToChunkedBuffer$1$adapted(SerializerHelper.scala:40)
	at org.apache.spark.serializer.SerializerHelper$$$Lambda$2795/0x00007feb7cdfb7d0.apply(Unknown Source)
	at org.apache.spark.util.io.ChunkedByteBufferOutputStream.allocateNewChunkIfNeeded(ChunkedByteBufferOutputStream.scala:87)
	at org.apache.spark.util.io.Chunk

ERROR:root:Exception while sending command.
Traceback (most recent call last):
  File "/usr/local/spark/python/lib/py4j-0.10.9.7-src.zip/py4j/clientserver.py", line 516, in send_command
    raise Py4JNetworkError("Answer from Java side is empty")
py4j.protocol.Py4JNetworkError: Answer from Java side is empty

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/local/spark/python/lib/py4j-0.10.9.7-src.zip/py4j/java_gateway.py", line 1038, in send_command
    response = connection.send_command(command)
               ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/spark/python/lib/py4j-0.10.9.7-src.zip/py4j/clientserver.py", line 539, in send_command
    raise Py4JNetworkError(
py4j.protocol.Py4JNetworkError: Error while sending or receiving


In [22]:
import os

os.environ['PYSPARK_SUBMIT_ARGS'] = '--packages org.apache.spark:spark-sql-kafka-0-10_2.12:3.5.2 pyspark-shell'

KAFKA_BOOTSTRAP_SERVERS = "kafka:9092"
KAFKA_TOPIC = "traffic_sensor"


df_traffic_stream.take(1)

df_traffic_stream\
    .writeStream\
    .format("kafka")\
    .option("kafka.bootstrap.servers", KAFKA_BOOTSTRAP_SERVERS)\
    .option("topic", KAFKA_TOPIC)\
    .option("checkpointLocation", "/tmp/checkpoint")\
    .start()\
    .awaitTermination()

AnalysisException: Queries with streaming sources must be executed with writeStream.start();
FileSource[work/output/parquet/*/*.parquet]

AnalysisException: Failed to find data source: kafka. Please deploy the application as per the deployment section of Structured Streaming + Kafka Integration Guide.